In [ ]:
import tensorflow as tf
from keras.preprocessing import image
from keras.utils import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import Sequential
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,BatchNormalization
from keras.utils import image_dataset_from_directory

In [ ]:
train_ds=image_dataset_from_directory(
    directory='/kaggle/input/apples-or-tomatoes-image-classification/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)
val_ds=image_dataset_from_directory(
    directory='/kaggle/input/apples-or-tomatoes-image-classification/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

In [ ]:
# create CNN model

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

# Methods improve performance of DL model

## Data scaling

In [ ]:
# # Normalize
# def process(image,label):
#     image = tf.cast(image/255. ,tf.float32)
#     return image,label

# train_ds = train_ds.map(process)
# val_ds = val_ds.map(process)

In [ ]:
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# history = model.fit(train_ds,epochs=10,validation_data=val_ds)

## Data Augmentation

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(
    '/kaggle/input/apples-or-tomatoes-image-classification/train',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)
val_generator=train_datagen.flow_from_directory(
    '/kaggle/input/apples-or-tomatoes-image-classification/test',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit_generator(train_generator,epochs=10,validation_data=val_generator)

## Early stopping

In [ ]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor="val_loss",           # Monitor validation loss for early stopping
    min_delta=0.00001,            # Minimum change in monitored quantity to qualify as an improvement
    patience=2,                   # Number of epochs with no improvement to wait before stopping
    verbose=1,                    # Print messages about early stopping to the console
    mode="auto",                  # Direction of improvement ("auto", "min", or "max")
    restore_best_weights=True    # Whether to restore model weights to the best epoch
)


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator,epochs=10,validation_data=val_generator,callbacks=callback)

## Transfer Learning

In [ ]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(256,256,3)
)
conv_base.trainable = False

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator,epochs=10,validation_data=val_generator)

## Fine Tuning

In [ ]:
for layer in conv_base.layers:
    if layer.name[0:6]=='block5':
        layer.trainable=True
    else:
        layer.trainable=False
for layer in conv_base.layers:
    print(layer.name,layer.trainable)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator,epochs=10,validation_data=val_generator)

## Chose Best optimizer

In [ ]:
model.compile(optimizer='AdaGrad',loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_generator,epochs=5,validation_data=val_generator)

In [ ]:
from tensorflow.keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator,epochs=5,validation_data=val_generator)